# Comparison of Warm vs Cold Starts

In [1]:
import asyncio
from sf_platform import ServerlessPlatform
import numpy as np
import pandas as pd

In [2]:
sp = ServerlessPlatform("./sf_platform/template/")

In [3]:
for i in range(10):
    await sp.register_function(f"cowsay-{i}", "./sf_platform/examples/cowsay/entry.py", "./sf_platform/examples/cowsay/requirements.txt")
    
    # Cold starts
    await asyncio.gather(*[sp.run_function(f"cowsay-{i}") for _ in range(10)])
    
    await asyncio.sleep(1)
    
    # Warm starts
    await asyncio.gather(*[sp.run_function(f"cowsay-{i}") for _ in range(10)])
    
    await asyncio.sleep(1)

pass # Supress notebook output

In [4]:
warm_start_durations = []
cold_start_durations = []
for row in sp.logs:
    if row['log_type'] != 'invocation_trace':
        continue
        
    duration = row['container_release_time'] - row['entry_time']
    
    if row['cold_start']:
        cold_start_durations.append(duration)
    else:
        warm_start_durations.append(duration)


### Warm Starts

In [5]:
pd.Series(np.array(warm_start_durations)).describe()

count    100.000000
mean       0.024760
std        0.007143
min        0.013962
25%        0.020440
50%        0.022773
75%        0.026528
max        0.044261
dtype: float64

### Cold Starts

In [6]:
pd.Series(np.array(cold_start_durations)).describe()

count    100.000000
mean       1.487672
std        0.703729
min        0.300435
25%        0.913092
50%        1.493160
75%        2.033018
max        3.301046
dtype: float64

In [7]:
sp.shutdown()